In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
print("⚠️  Grid-wide sync is a CUDA C++ feature requiring cooperative launch!")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

## Part 1: Why Grid-Wide Sync?

### The Problem

```
Traditional CUDA - Multiple Kernel Launches:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

kernel1<<<blocks, threads>>>(data);  // Pass 1
cudaDeviceSynchronize();              // Host overhead!
kernel2<<<blocks, threads>>>(data);  // Pass 2
cudaDeviceSynchronize();              // Host overhead!
kernel3<<<blocks, threads>>>(data);  // Pass 3

Problems:
• Each launch has ~5-15μs overhead
• Data goes through L2 cache multiple times
• Can't keep data in L1/registers between passes

With Grid Sync - Single Kernel:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

__global__ void fusedKernel(data) {
    // Pass 1
    process1(data);
    grid.sync();      // GPU-only sync, ~1μs!
    
    // Pass 2
    process2(data);
    grid.sync();
    
    // Pass 3
    process3(data);
}
```

---

## Part 2: Basic Grid Synchronization

### 🔷 CUDA C++ Grid Group (Primary)

Compile: `nvcc -arch=sm_75 -o grid_sync grid_sync.cu`

In [ ]:
%%writefile grid_sync.cu
// grid_sync.cu - Basic grid-wide synchronization
#include <stdio.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h>

namespace cg = cooperative_groups;

__global__ void gridSyncDemo(float* data, int n) {
    // ============================================
    // Get Grid Group
    // ============================================
    cg::grid_group grid = cg::this_grid();
    
    int tid = grid.thread_rank();  // Global thread index
    int gridSize = grid.size();    // Total threads in grid
    
    if (tid == 0) {
        printf("Grid has %d threads\n", gridSize);
    }
    
    // Pass 1: Multiply
    if (tid < n) {
        data[tid] *= 2.0f;
    }
    
    // ============================================
    // Grid-Wide Synchronization
    // ============================================
    grid.sync();  // ALL blocks wait here!
    
    // Pass 2: Add (safe - all threads done with pass 1)
    if (tid < n) {
        data[tid] += 1.0f;
    }
    
    grid.sync();  // Sync again
    
    // Pass 3: Sqrt
    if (tid < n) {
        data[tid] = sqrtf(data[tid]);
    }
}

int main() {
    const int N = 1 << 20;
    const int BLOCK_SIZE = 256;
    
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // Initialize
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) h_data[i] = 4.0f;
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    
    // ============================================
    // Query Maximum Blocks for Cooperative Launch
    // ============================================
    int device = 0;
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, device);
    
    int numBlocksPerSM;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(
        &numBlocksPerSM, gridSyncDemo, BLOCK_SIZE, 0);
    
    int maxBlocks = numBlocksPerSM * prop.multiProcessorCount;
    int numBlocks = min(maxBlocks, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);
    
    printf("Using %d blocks (max %d)\n", numBlocks, maxBlocks);
    
    // ============================================
    // Cooperative Kernel Launch
    // ============================================
    void* args[] = { &d_data, (void*)&N };
    
    cudaLaunchCooperativeKernel(
        (void*)gridSyncDemo,
        dim3(numBlocks),
        dim3(BLOCK_SIZE),
        args,
        0,       // Shared memory
        0        // Stream (default)
    );
    
    cudaDeviceSynchronize();
    
    // Verify
    cudaMemcpy(h_data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost);
    printf("Result[0] = %.4f (expected %.4f)\n", h_data[0], sqrtf(4.0f * 2.0f + 1.0f));
    
    delete[] h_data;
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o grid_sync grid_sync.cu
!./grid_sync

---

## Part 3: Occupancy Requirements

### Why Limit Grid Size?

```
Grid Sync Requirement:
━━━━━━━━━━━━━━━━━━━━━━━

ALL blocks must be resident simultaneously!

If you launch too many blocks:
• Some blocks can't start
• Running blocks wait at sync()
• Waiting blocks never start
• DEADLOCK!

Solution: Query max occupancy

cudaOccupancyMaxActiveBlocksPerMultiprocessor(&blocksPerSM, kernel, blockSize, sharedMem);
maxBlocks = blocksPerSM × numSMs;
```

### 🔷 CUDA C++ Occupancy Calculator (Primary)

In [ ]:
%%writefile occupancy_check.cu
// occupancy_check.cu - Calculate safe grid size

#include <cuda_runtime.h>
#include <cooperative_groups.h>
#include <stdio.h>

namespace cg = cooperative_groups;

__global__ void cooperativeKernel(float* data, int n) {
    cg::grid_group grid = cg::this_grid();
    // ... work ...
    grid.sync();
}

int getMaxCooperativeBlocks(int blockSize, size_t sharedMem = 0) {
    int device;
    cudaGetDevice(&device);
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, device);
    
    // Check if cooperative launch is supported
    if (!prop.cooperativeLaunch) {
        printf("Cooperative launch not supported!\n");
        return 0;
    }
    
    int blocksPerSM;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(
        &blocksPerSM, 
        cooperativeKernel, 
        blockSize, 
        sharedMem);
    
    return blocksPerSM * prop.multiProcessorCount;
}

int main() {
    printf("Max cooperative blocks (256 threads): %d\n", getMaxCooperativeBlocks(256));
    printf("Max cooperative blocks (512 threads): %d\n", getMaxCooperativeBlocks(512));
    printf("Max cooperative blocks (1024 threads): %d\n", getMaxCooperativeBlocks(1024));
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o occupancy_check occupancy_check.cu
!./occupancy_check

---

## Part 4: Practical Example - Multi-Pass Reduction

### 🔷 CUDA C++ Grid Reduction (Primary)

In [ ]:
%%writefile grid_reduction.cu
// grid_reduction.cu - Single-kernel full reduction
#include <stdio.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h>

namespace cg = cooperative_groups;

__device__ float warpReduce(cg::thread_block_tile<32>& warp, float val) {
    for (int i = warp.size()/2; i > 0; i /= 2) {
        val += warp.shfl_down(val, i);
    }
    return val;
}

__global__ void gridReduce(float* input, float* output, int n) {
    cg::grid_group grid = cg::this_grid();
    cg::thread_block block = cg::this_thread_block();
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    extern __shared__ float sdata[];
    
    int tid = grid.thread_rank();
    int gridSize = grid.size();
    
    // ============================================
    // Phase 1: Grid-stride loop to load & reduce
    // ============================================
    float sum = 0.0f;
    for (int i = tid; i < n; i += gridSize) {
        sum += input[i];
    }
    
    // Warp reduction
    sum = warpReduce(warp, sum);
    
    // Store warp results to shared memory
    int lane = warp.thread_rank();
    int warp_id = block.thread_rank() / 32;
    if (lane == 0) {
        sdata[warp_id] = sum;
    }
    block.sync();
    
    // First warp reduces shared memory
    int numWarps = (block.size() + 31) / 32;
    sum = (block.thread_rank() < numWarps) ? sdata[block.thread_rank()] : 0.0f;
    if (warp_id == 0) {
        sum = warpReduce(warp, sum);
    }
    
    // Block leader writes to global array
    if (block.thread_rank() == 0) {
        output[blockIdx.x] = sum;
    }
    
    // ============================================
    // Grid Sync - Wait for all blocks
    // ============================================
    grid.sync();
    
    // ============================================
    // Phase 2: Final reduction (first block only)
    // ============================================
    if (blockIdx.x == 0) {
        int numBlocks = gridDim.x;
        sum = 0.0f;
        
        // Each thread sums subset of block results
        for (int i = block.thread_rank(); i < numBlocks; i += block.size()) {
            sum += output[i];
        }
        
        // Reduce within block
        sum = warpReduce(warp, sum);
        if (lane == 0) sdata[warp_id] = sum;
        block.sync();
        
        sum = (block.thread_rank() < numWarps) ? sdata[block.thread_rank()] : 0.0f;
        if (warp_id == 0) sum = warpReduce(warp, sum);
        
        // Final result
        if (block.thread_rank() == 0) {
            output[0] = sum;
        }
    }
}

int main() {
    const int N = 1 << 24;  // 16M elements
    const int BLOCK_SIZE = 256;
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, 1024 * sizeof(float));  // Workspace
    
    // Initialize with 1s
    float* h_input = new float[N];
    for (int i = 0; i < N; i++) h_input[i] = 1.0f;
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    
    // Get max blocks
    int device;
    cudaGetDevice(&device);
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, device);
    
    int blocksPerSM;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(
        &blocksPerSM, gridReduce, BLOCK_SIZE, 32 * sizeof(float));
    int numBlocks = blocksPerSM * prop.multiProcessorCount;
    
    printf("Launching %d blocks\n", numBlocks);
    
    // Launch
    void* args[] = { &d_input, &d_output, (void*)&N };
    cudaLaunchCooperativeKernel(
        (void*)gridReduce,
        dim3(numBlocks),
        dim3(BLOCK_SIZE),
        args,
        32 * sizeof(float),
        0);
    
    float result;
    cudaMemcpy(&result, d_output, sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("Sum = %.0f (expected %d)\n", result, N);
    
    delete[] h_input;
    cudaFree(d_input);
    cudaFree(d_output);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o grid_reduction grid_reduction.cu
!./grid_reduction

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile grid_sync_exercises.cu
/*
 * Grid-Wide Synchronization Exercises
 * Exercise 1: Grid Prefix Sum - Blelloch scan with grid sync
 * Exercise 2: Iterative Solver - Jacobi iteration with grid sync
 * Exercise 3: Histogram with Grid Sync - Global histogram in single kernel
 */

#include <cuda_runtime.h>
#include <cooperative_groups.h>
#include <stdio.h>
#include <cmath>

namespace cg = cooperative_groups;

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// =============================================================
// Exercise 1: Grid Prefix Sum (Inclusive Scan)
// Multi-phase scan using grid synchronization
// =============================================================
__global__ void gridPrefixSumKernel(int* data, int* blockSums, int n) {
    cg::grid_group grid = cg::this_grid();
    cg::thread_block block = cg::this_thread_block();
    
    extern __shared__ int sdata[];
    
    int tid = threadIdx.x;
    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Phase 1: Load data into shared memory
    sdata[tid] = (globalIdx < n) ? data[globalIdx] : 0;
    block.sync();
    
    // Phase 2: Block-level inclusive scan (Hillis-Steele)
    for (int stride = 1; stride < blockDim.x; stride *= 2) {
        int temp = 0;
        if (tid >= stride) {
            temp = sdata[tid - stride];
        }
        block.sync();
        sdata[tid] += temp;
        block.sync();
    }
    
    // Store block sum (last element of each block's scan)
    if (tid == blockDim.x - 1) {
        blockSums[blockIdx.x] = sdata[tid];
    }
    
    // Write partial results
    if (globalIdx < n) {
        data[globalIdx] = sdata[tid];
    }
    
    // Grid sync: wait for all blocks to finish Phase 2
    grid.sync();
    
    // Phase 3: Scan the block sums (only block 0 does this)
    if (blockIdx.x == 0) {
        // Load block sums into shared memory
        if (tid < gridDim.x) {
            sdata[tid] = blockSums[tid];
        } else {
            sdata[tid] = 0;
        }
        block.sync();
        
        // Scan block sums
        for (int stride = 1; stride < gridDim.x; stride *= 2) {
            int temp = 0;
            if (tid >= stride && tid < gridDim.x) {
                temp = sdata[tid - stride];
            }
            block.sync();
            if (tid < gridDim.x) sdata[tid] += temp;
            block.sync();
        }
        
        // Store scanned block sums
        if (tid < gridDim.x) {
            blockSums[tid] = sdata[tid];
        }
    }
    
    // Grid sync: wait for block sums to be scanned
    grid.sync();
    
    // Phase 4: Add scanned block sums to elements
    if (blockIdx.x > 0 && globalIdx < n) {
        data[globalIdx] += blockSums[blockIdx.x - 1];
    }
}

void exercise1_grid_prefix_sum() {
    printf("\n=== Exercise 1: Grid Prefix Sum ===\n");
    
    // Check for cooperative launch support
    int device;
    cudaDeviceProp prop;
    CHECK_CUDA(cudaGetDevice(&device));
    CHECK_CUDA(cudaGetDeviceProperties(&prop, device));
    
    if (!prop.cooperativeLaunch) {
        printf("Device does not support cooperative launch. Skipping.\n");
        return;
    }
    
    const int BLOCK_SIZE = 256;
    const int N = BLOCK_SIZE * 4;  // 4 blocks worth
    
    int *h_data = new int[N];
    int *h_result = new int[N];
    int *h_expected = new int[N];
    int *d_data, *d_blockSums;
    
    // Initialize: all ones (prefix sum should give 1, 2, 3, ...)
    for (int i = 0; i < N; i++) {
        h_data[i] = 1;
    }
    
    // Compute expected result
    h_expected[0] = h_data[0];
    for (int i = 1; i < N; i++) {
        h_expected[i] = h_expected[i-1] + h_data[i];
    }
    
    int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(int)));
    CHECK_CUDA(cudaMalloc(&d_blockSums, numBlocks * sizeof(int)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, N * sizeof(int), cudaMemcpyHostToDevice));
    
    // Launch cooperative kernel
    void* kernelArgs[] = { &d_data, &d_blockSums, (void*)&N };
    size_t sharedMem = BLOCK_SIZE * sizeof(int);
    
    CHECK_CUDA(cudaLaunchCooperativeKernel(
        (void*)gridPrefixSumKernel,
        dim3(numBlocks), dim3(BLOCK_SIZE),
        kernelArgs, sharedMem, 0
    ));
    CHECK_CUDA(cudaDeviceSynchronize());
    
    CHECK_CUDA(cudaMemcpy(h_result, d_data, N * sizeof(int), cudaMemcpyDeviceToHost));
    
    // Verify
    printf("Array size: %d, Blocks: %d\n", N, numBlocks);
    printf("\nSample results (every 64th element):\n");
    printf("%-8s %-10s %-10s\n", "Index", "Result", "Expected");
    
    bool correct = true;
    for (int i = 0; i < N; i += 64) {
        printf("%-8d %-10d %-10d\n", i, h_result[i], h_expected[i]);
        if (h_result[i] != h_expected[i]) correct = false;
    }
    
    // Also check boundaries
    for (int i = 0; i < N; i++) {
        if (h_result[i] != h_expected[i]) correct = false;
    }
    
    printf("\nVerification: %s\n", correct ? "PASSED ✓" : "FAILED ✗");
    
    delete[] h_data;
    delete[] h_result;
    delete[] h_expected;
    CHECK_CUDA(cudaFree(d_data));
    CHECK_CUDA(cudaFree(d_blockSums));
}

// =============================================================
// Exercise 2: Jacobi Iteration with Grid Sync
// Iterative solver that alternates between arrays
// =============================================================
__global__ void jacobiIterationKernel(float* u, float* u_new, float* errors, 
                                       int n, int numIterations) {
    cg::grid_group grid = cg::this_grid();
    
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Pointers for double-buffering
    float* src = u;
    float* dst = u_new;
    
    for (int iter = 0; iter < numIterations; iter++) {
        // Update interior points: u_new[i] = 0.5 * (u[i-1] + u[i+1])
        if (idx > 0 && idx < n - 1) {
            dst[idx] = 0.5f * (src[idx - 1] + src[idx + 1]);
        } else if (idx == 0 || idx == n - 1) {
            dst[idx] = src[idx];  // Boundary conditions
        }
        
        // Grid sync before computing errors
        grid.sync();
        
        // Compute local error
        if (idx < n) {
            errors[idx] = fabsf(dst[idx] - src[idx]);
        }
        
        // Grid sync before next iteration
        grid.sync();
        
        // Swap buffers
        float* temp = src;
        src = dst;
        dst = temp;
    }
    
    // Copy final result to u if needed
    if (numIterations % 2 == 1 && idx < n) {
        u[idx] = u_new[idx];
    }
}

void exercise2_jacobi_iteration() {
    printf("\n=== Exercise 2: Jacobi Iteration ===\n");
    
    int device;
    cudaDeviceProp prop;
    CHECK_CUDA(cudaGetDevice(&device));
    CHECK_CUDA(cudaGetDeviceProperties(&prop, device));
    
    if (!prop.cooperativeLaunch) {
        printf("Device does not support cooperative launch. Skipping.\n");
        return;
    }
    
    const int N = 256;
    const int BLOCK_SIZE = 256;
    const int NUM_ITERATIONS = 100;
    
    float *h_u = new float[N];
    float *h_result = new float[N];
    float *d_u, *d_u_new, *d_errors;
    
    // Initialize: step function (boundary value problem)
    for (int i = 0; i < N; i++) {
        if (i < N / 2) {
            h_u[i] = 0.0f;
        } else {
            h_u[i] = 1.0f;
        }
    }
    h_u[0] = 0.0f;      // Left boundary
    h_u[N-1] = 1.0f;    // Right boundary
    
    printf("Solving 1D Laplace equation with Jacobi iteration\n");
    printf("Domain size: %d, Iterations: %d\n", N, NUM_ITERATIONS);
    
    CHECK_CUDA(cudaMalloc(&d_u, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_u_new, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_errors, N * sizeof(float)));
    CHECK_CUDA(cudaMemcpy(d_u, h_u, N * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_u_new, h_u, N * sizeof(float), cudaMemcpyHostToDevice));
    
    int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Launch cooperative kernel
    void* kernelArgs[] = { &d_u, &d_u_new, &d_errors, (void*)&N, (void*)&NUM_ITERATIONS };
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    CHECK_CUDA(cudaEventRecord(start));
    CHECK_CUDA(cudaLaunchCooperativeKernel(
        (void*)jacobiIterationKernel,
        dim3(numBlocks), dim3(BLOCK_SIZE),
        kernelArgs, 0, 0
    ));
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    
    float gpuTime;
    CHECK_CUDA(cudaEventElapsedTime(&gpuTime, start, stop));
    
    CHECK_CUDA(cudaMemcpy(h_result, d_u, N * sizeof(float), cudaMemcpyDeviceToHost));
    
    // Expected: linear interpolation from 0 to 1
    printf("\nSolution (sampled at 8 points):\n");
    printf("%-8s %-12s %-12s\n", "Index", "Computed", "Analytical");
    for (int i = 0; i < N; i += N/8) {
        float analytical = (float)i / (N - 1);
        printf("%-8d %-12.4f %-12.4f\n", i, h_result[i], analytical);
    }
    
    // Check convergence to analytical solution
    float maxError = 0.0f;
    for (int i = 0; i < N; i++) {
        float analytical = (float)i / (N - 1);
        float error = fabsf(h_result[i] - analytical);
        maxError = fmaxf(maxError, error);
    }
    
    printf("\nMax error from analytical solution: %.6f\n", maxError);
    printf("GPU time: %.3f ms (%.1f iterations/ms)\n", gpuTime, NUM_ITERATIONS / gpuTime);
    printf("Verification: %s\n", maxError < 0.01f ? "PASSED ✓" : "FAILED ✗");
    
    delete[] h_u;
    delete[] h_result;
    CHECK_CUDA(cudaEventDestroy(start));
    CHECK_CUDA(cudaEventDestroy(stop));
    CHECK_CUDA(cudaFree(d_u));
    CHECK_CUDA(cudaFree(d_u_new));
    CHECK_CUDA(cudaFree(d_errors));
}

// =============================================================
// Exercise 3: Histogram with Grid Sync
// Build global histogram in a single kernel using grid sync
// =============================================================
#define NUM_BINS 256

__global__ void gridHistogramKernel(unsigned char* data, int* histogram, 
                                     int* blockHistograms, int n) {
    cg::grid_group grid = cg::this_grid();
    cg::thread_block block = cg::this_thread_block();
    
    // Each block has its own shared histogram
    __shared__ int sharedHist[NUM_BINS];
    
    // Initialize shared histogram
    for (int i = threadIdx.x; i < NUM_BINS; i += blockDim.x) {
        sharedHist[i] = 0;
    }
    block.sync();
    
    // Phase 1: Compute block-local histograms
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    for (int i = idx; i < n; i += blockDim.x * gridDim.x) {
        atomicAdd(&sharedHist[data[i]], 1);
    }
    block.sync();
    
    // Store block histogram to global memory
    for (int i = threadIdx.x; i < NUM_BINS; i += blockDim.x) {
        blockHistograms[blockIdx.x * NUM_BINS + i] = sharedHist[i];
    }
    
    // Grid sync: wait for all block histograms
    grid.sync();
    
    // Phase 2: Reduce block histograms to global histogram
    // Each thread handles one or more bins
    for (int bin = threadIdx.x + blockIdx.x * blockDim.x; 
         bin < NUM_BINS; 
         bin += blockDim.x * gridDim.x) {
        
        int sum = 0;
        for (int b = 0; b < gridDim.x; b++) {
            sum += blockHistograms[b * NUM_BINS + bin];
        }
        histogram[bin] = sum;
    }
}

void exercise3_histogram_grid_sync() {
    printf("\n=== Exercise 3: Histogram with Grid Sync ===\n");
    
    int device;
    cudaDeviceProp prop;
    CHECK_CUDA(cudaGetDevice(&device));
    CHECK_CUDA(cudaGetDeviceProperties(&prop, device));
    
    if (!prop.cooperativeLaunch) {
        printf("Device does not support cooperative launch. Skipping.\n");
        return;
    }
    
    const int N = 1024 * 1024;
    const int BLOCK_SIZE = 256;
    
    unsigned char *h_data = new unsigned char[N];
    int *h_histogram = new int[NUM_BINS];
    int *h_reference = new int[NUM_BINS];
    
    unsigned char *d_data;
    int *d_histogram, *d_blockHistograms;
    
    // Initialize with random data
    srand(42);
    for (int i = 0; i < N; i++) {
        h_data[i] = rand() % 256;
    }
    
    // CPU reference
    memset(h_reference, 0, NUM_BINS * sizeof(int));
    for (int i = 0; i < N; i++) {
        h_reference[h_data[i]]++;
    }
    
    // Determine launch configuration
    int numBlocksPerSM;
    CHECK_CUDA(cudaOccupancyMaxActiveBlocksPerMultiprocessor(
        &numBlocksPerSM, gridHistogramKernel, BLOCK_SIZE, 0));
    int numBlocks = prop.multiProcessorCount * numBlocksPerSM;
    
    printf("Data size: %d elements\n", N);
    printf("Using %d blocks (%d blocks/SM × %d SMs)\n", 
           numBlocks, numBlocksPerSM, prop.multiProcessorCount);
    
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_histogram, NUM_BINS * sizeof(int)));
    CHECK_CUDA(cudaMalloc(&d_blockHistograms, numBlocks * NUM_BINS * sizeof(int)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, N, cudaMemcpyHostToDevice));
    
    // Launch cooperative kernel
    void* kernelArgs[] = { &d_data, &d_histogram, &d_blockHistograms, (void*)&N };
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    CHECK_CUDA(cudaEventRecord(start));
    CHECK_CUDA(cudaLaunchCooperativeKernel(
        (void*)gridHistogramKernel,
        dim3(numBlocks), dim3(BLOCK_SIZE),
        kernelArgs, 0, 0
    ));
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    
    float gpuTime;
    CHECK_CUDA(cudaEventElapsedTime(&gpuTime, start, stop));
    
    CHECK_CUDA(cudaMemcpy(h_histogram, d_histogram, NUM_BINS * sizeof(int), 
                          cudaMemcpyDeviceToHost));
    
    // Verify
    bool correct = true;
    int totalCount = 0;
    for (int i = 0; i < NUM_BINS; i++) {
        if (h_histogram[i] != h_reference[i]) {
            correct = false;
            printf("Mismatch at bin %d: got %d, expected %d\n", 
                   i, h_histogram[i], h_reference[i]);
        }
        totalCount += h_histogram[i];
    }
    
    printf("\nHistogram statistics:\n");
    printf("  Total elements counted: %d (expected %d)\n", totalCount, N);
    printf("  Non-zero bins: ");
    int nonZeroBins = 0;
    for (int i = 0; i < NUM_BINS; i++) {
        if (h_histogram[i] > 0) nonZeroBins++;
    }
    printf("%d\n", nonZeroBins);
    
    printf("\nGPU time: %.3f ms\n", gpuTime);
    printf("Throughput: %.2f GB/s\n", (N / 1e9) / (gpuTime / 1000.0));
    printf("Verification: %s\n", correct ? "PASSED ✓" : "FAILED ✗");
    
    delete[] h_data;
    delete[] h_histogram;
    delete[] h_reference;
    CHECK_CUDA(cudaEventDestroy(start));
    CHECK_CUDA(cudaEventDestroy(stop));
    CHECK_CUDA(cudaFree(d_data));
    CHECK_CUDA(cudaFree(d_histogram));
    CHECK_CUDA(cudaFree(d_blockHistograms));
}

int main() {
    printf("Grid-Wide Synchronization Exercises\n");
    printf("====================================\n");
    
    int device;
    cudaDeviceProp prop;
    CHECK_CUDA(cudaGetDevice(&device));
    CHECK_CUDA(cudaGetDeviceProperties(&prop, device));
    printf("Device: %s (Compute %d.%d)\n", prop.name, prop.major, prop.minor);
    printf("Cooperative launch support: %s\n", 
           prop.cooperativeLaunch ? "Yes" : "No");
    
    exercise1_grid_prefix_sum();
    exercise2_jacobi_iteration();
    exercise3_histogram_grid_sync();
    
    printf("\n✓ All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -o grid_sync_exercises grid_sync_exercises.cu -lcudadevrt && ./grid_sync_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Grid Prefix Sum
Implement Blelloch scan using grid sync for multiple phases.

### Exercise 2: Iterative Solver
Implement Jacobi iteration with grid sync between iterations.

### Exercise 3: Histogram with Grid Sync
Build a global histogram in a single kernel using grid sync.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────┐
│           GRID-WIDE SYNCHRONIZATION                     │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Get Grid Group:                                        │
│  cg::grid_group grid = cg::this_grid();                 │
│                                                         │
│  Synchronize:                                           │
│  grid.sync();  // All blocks wait here                  │
│                                                         │
│  Launch Requirements:                                   │
│  • Use cudaLaunchCooperativeKernel()                    │
│  • Limit blocks to max occupancy                        │
│  • Check prop.cooperativeLaunch                         │
│                                                         │
│  Benefits:                                              │
│  • Eliminate kernel launch overhead                     │
│  • Keep data in cache between phases                    │
│  • Single kernel for multi-pass algorithms              │
│                                                         │
│  Limitations:                                           │
│  • Grid size limited by occupancy                       │
│  • Requires compute capability 3.5+                     │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

## Next: Day 3 - Dynamic Parallelism Basics